# Introduction

The MultiViz Vibration Service is an analytics service based on the MultiViz Vibration Engine.
To access it Viking Analytics provides a REST API towards its analytics server.
To simplify the access of the Vibration service, Viking Analytics provides a Python Package
called VPP, which allows to interact with the vibration service via regular python calls.

This interactive document shall show the whole flow working with the vibration service from
Python, from data upload to retrieval of analysis results.

## Signing up for the serivce

For commercial use of the services you will need to acquire a token from Viking Analytics.
For the example here, we can use a built in token for demo, we also use a special
URL for demo cases.

## Python Pre-requisites

1. Make sure you have Python 3.6 or higher installed.
2. Install the VPP package from TBD
3. Make sure you have the following packages installed (json, matplotlib...)

In the follwing section we will walk through the code needed for the API interaction


### Setting up URL and Token

For any call to the API, you will need to provide the server URL ('endpoint') and the 
token. To simplify this interaction the VPP library defines a class VPP, by means of
which you can instantiate a session object chaching URL and token and providing methods
for the API call.


In [1]:
import vpp.vpp_analysis_stub as vpp
session = vpp.VPP_analysis_stub("http://127.0.0.1:8000", "NO TOKEN")

The ´session´ object now caches the URL and token and is ready to interact
with the analytics service

### Checking if the service is alive

To check if the service is alive, we will query for information about the API.

In [4]:
session.say_hello()

{'message': 'Vibium App!'}

If everything goes well, a dictionary with the proudct name and the API version shall be returned.

### Creating a Source

A source represents a vibration data source, typically a vibration sensor. Internally in the
analytics engine and the anaytics data storage, all vibration data is stored under its source. 
A source basically is an ID, the source ID and meta data with a few required fields (TBD not yet)
and the possibility to store arbitrary customer specific 'free form' data along the source. The vibration service
will only rely on the required fields, the free form data is a possibility for the client side to 
keep together source information, measurements and meta data which may be interesting for
the analytics features built by means of the service. Example could be location of sensor, the asset 
which it is mounted on etc.
As we will see later, timestamps are internally represented as millisconds since EPOCH (Jan 1st 1970),
for that reason it is good practice to include the timezone where the measurement originated in 
the meta data.

In [3]:
source_id = "my_first_source"
meta_information = {'location': 'Molnlycke', 'assed ID': 't-55', 'timezone': 'Europe/Stockholm'}
session.create_source(source_id,meta_information)

The call returns silently, unless the resource could not be successfully created. Just to show
what happens in the latter case, lets pass a malformed call lacking the meta information.

In [4]:
session.create_source(source_id)

TypeError: create_source() missing 1 required positional argument: 'meta'

It is the caller's responsibility to handle error conditions. When something goes wrong expections will be raised,
either by the type checking mechanism in the VPP library (as above) or by the analytics server. In the latter 
caseHTTP response status codes will be returned; a list can be found [here](https://developer.mozilla.org/sv-SE/docs/Web/HTTP/Status).
So let's try to catch the above error:


In [5]:
try:
    session.create_source(source_id)
except:
    print("Something went wrong")

Something went wrong


### List sources
We can now check if our source actually has been created, by listing all the sources in the database.
TBD: wrong response from server (fixed in Mohsen's PR)

In [6]:
sources = session.list_sources()
print(f"Retrieved a total of {len(sources)} sources")

Retrieved a total of 467 sources


In [7]:
sources

[{'source_id': 'u0001',
  'timestamp': 1570186860,
  'duration': 40000,
  'meta': {'sensor_type': 'mfpt'}},
 {'source_id': 'u0001',
  'timestamp': 1570273260,
  'duration': 40000,
  'meta': {'sensor_type': 'mfpt'}},
 {'source_id': 'u0001',
  'timestamp': 1570359660,
  'duration': 40000,
  'meta': {'sensor_type': 'mfpt'}},
 {'source_id': 'u0001',
  'timestamp': 1570446060,
  'duration': 40000,
  'meta': {'sensor_type': 'mfpt'}},
 {'source_id': 'u0001',
  'timestamp': 1570532460,
  'duration': 40000,
  'meta': {'sensor_type': 'mfpt'}},
 {'source_id': 'u0001',
  'timestamp': 1570618860,
  'duration': 40000,
  'meta': {'sensor_type': 'mfpt'}},
 {'source_id': 'u0001',
  'timestamp': 1570705260,
  'duration': 40000,
  'meta': {'sensor_type': 'mfpt'}},
 {'source_id': 'u0001',
  'timestamp': 1570791660,
  'duration': 40000,
  'meta': {'sensor_type': 'mfpt'}},
 {'source_id': 'u0001',
  'timestamp': 1570878060,
  'duration': 40000,
  'meta': {'sensor_type': 'mfpt'}},
 {'source_id': 'u0001',
  't

## Upload measurements

Now that we have created a source, we can upload vibration measurements related to the source. To be processed by the analytics engine, the following requirements need to be met for the upload

#### Meta-data: 

The meta data needs to contain the source id ('my_first_source' in our case), to associate the measurement to the source it orginated from, the timestamp when the measurement was recorded (as an integer representing the millisecongs since EPOCH), the duration (in seconds as float value), to do a correct conversion to the frequency domain. For later use on the client side, more meta information can be added, but it will not be processed by the analytics engine.

#### Raw Data Values

The raw data values shall be provided as list of floating point values. Raw data values need to represent a time-domain samples.

#### Recommendations

The anlytics engine is agnostic to the scale nature and scale of the Raw Data values. We recommend to use acceleration measured in g. But the algorithm will work equally well with other vibration like data as velocities or sound files in any arbitrary scale. The only imporant point is that the duration given in the meta data actually reflects the sampling time and that sampling rate under this period is constant. If this is not assured, the frequency conversion will have undefined results, and the output of the algorithm cannot be interpreted.
Also it is required that a time series of measurements under one source use the sampe sampling rate.


## Analysis

List features

Request (call) analysis

Get analysis

## Display results

Plot mode changes

Read measurements and map them to modes

Display basic statistics on each mode

Results available:
- Mode table
- Labels table
- output.json with experiment information
- process_data (kpis & bands)